### 1. Content Based Recommender System 

We will try to build a system that recommends recipes that are similar to a particular recipe. We will use recipe ingredients and categories to determine similarity and then list the top 10 closest matches.

In [403]:
%load_ext Cython
%time
%cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


UsageError: Line magic function `%cython` not found (But cell magic `%%cython` exists, did you mean that instead?).


In [404]:
#import needed packages
from gensim.models import doc2vec

import re
import pandas as pd
import numpy as np
import copy

# nltk processing
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier

In [405]:
# read in fields title, ingredients and categories
data = pd.read_csv("dataRecTitleCatsIngred.csv") 

data.duplicated()


0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
8080    False
8081    False
8082    False
8083    False
8084    False
8085    False
8086    False
8087     True
8088    False
8089    False
8090    False
8091    False
8092    False
8093    False
8094    False
8095    False
8096    False
8097    False
8098    False
8099    False
8100    False
8101    False
8102    False
8103    False
8104    False
8105    False
8106    False
8107    False
8108    False
8109    False
Length: 8110, dtype: bool

In [406]:
data.drop_duplicates()

,title,categories,ingred
0,1-Dish Taco Bake,"['Trusted Brands: Recipes and Tips', ""ARGO&#17...","['Taco Meat Filling:', '1 pound ground beef', ..."
1,15-Minute Dinner Nachos Supreme,"['Trusted Brands: Recipes and Tips', ""Campbell...","['1 pound ground beef', '1 (1.12 ounce) packag..."
2,A Firefighter's Meatloaf,"['Main Dish', 'Meatloaf', 'Beef Meatloaf']","['2 slices whole wheat bread', '2 pounds groun..."
3,Aaron's Missouri Burger,"['Main Dish', 'Burgers', 'Hamburgers']","['1 pound lean ground beef', '2 teaspoons onio..."
4,Absolute Best Liver and Onions,['Main Dish'],"['2 pounds sliced beef liver', '1 1/2 cups mil..."
5,Adobo Sirloin,"['World Cuisine', 'Latin American', 'Mexican']","['1 lime, juiced', '1 tablespoon minced garlic..."
6,Albondigas,"['Soups, Stews and Chili', 'Soup', 'Beef Soup']","['1 quart water', '4 carrots, sliced', '2 smal..."
7,Albondigas Soup I,"['Soups, Stews and Chili', 'Soup', 'Pork Soup']","['1 pound lean ground beef', '1/4 pound pork s..."
8,Albondigas Soup II,"['Soups, Stews and Chili', 'Soup', 'Beef Soup']","['1 pound lean ground beef', '2 cubes beef bou..."
9,Alison's Slow Cooker Vegetable Beef Soup,"['Soups, Stews and Chili', 'Soup', 'Beef Soup']","['1 1/2 pounds cubed beef stew meat', '2 cups ..."


In [407]:
### removed measurements, descriptive words and anything that's not an ingredient
def clean_data_ingred(ds):
    arr = ['tablespoons','tablespoon','teaspoons','teaspoon','cups','cup','ounces','ounce','pounds','pound','pinches','pinch','tastes','taste','dashes','dash','/',' or ',' as ',' and ',
          '-','[',']','(',')','.','slices','sliced','slice', '0','1','2','3','4','5','6','7','8','9','juiced','diced','minced','chopped',' cans',' canned ',' can ', ' inches ', ' inch ',
          'package', 'finely', 'crushed', 'shredded', 'divided', ' melted ', ' jar ','thawed', ' cut ', ' to ', ' thick ', ' across ', ' bones ', 'optional', ' split ', ' torn ',
          ' into ', ' strips ', 'peeled',' pieces', ' lightly ', ' beaten ','ground black', 'scrubbed', 'jars', 'beaten','pitted','halved', 'thinly','as needed','large','medium','small'
          'roughly','fluid','drained','crumbled','rinsed','cubed','seeded','fresh',' plus ', ' more ', 'deboned', 'prepared', 'cubes', 'uncooked', 'cooked', 'grated', ' lean ','into chunks',
          'coarsely','seeded','freshly',' freshly', 'fresh', ' with ', 'liquid','quartered','lengthwise','small',' into ',' bite ', ' sized ', ' ly ', ' ed ', ' s ','cut into', ' up ',
          'trimmed','frozen','chunks','segments','softened',' cut ','pieces','broken',' rings',' torn ', ' ',"'"]

    ### merge multiple words, for example 'ground beef' becomes 'groundbeef'
    for item in arr:
        ds = ds.str.replace(item,'')
     
    return ds

In [408]:
### Remove measurements and anything else that is not a food ingredient
data['ingred'] = clean_data_ingred(data['ingred'])

#data['title'] = clean_data(data['title'])
#dfIngred.to_csv('ingredClean.csv', sep=' ')

data['ingred'].head()

0    TacoMeatFilling:,groundbeef,tacoseasoning,Batt...
1    groundbeef,Pace®TacoSeasoningMix,"Campbells®Co...
2    wholewheatbread,groundbeef,greenonion,,onion,s...
3    groundbeef,onionpowder,honeymustard,garlicpowd...
4    beefliver,milk,,butter,,Vidaliaonions,,allpurp...
Name: ingred, dtype: object

In [409]:
### separate each term with a space
def create_soup(ds):
    ds = ds.str.replace(',',' ')
    return ds

In [410]:
### Separate each ingredient with a space
data['ingredSpaces'] = create_soup(data['ingred'])
data['ingredSpaces'].head()

0    TacoMeatFilling: groundbeef tacoseasoning Batt...
1    groundbeef Pace®TacoSeasoningMix "Campbells®Co...
2    wholewheatbread groundbeef greenonion  onion s...
3    groundbeef onionpowder honeymustard garlicpowd...
4    beefliver milk  butter  Vidaliaonions  allpurp...
Name: ingredSpaces, dtype: object

In [411]:
### Compute Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each recipe
### This will give us a matrix where each column represents a word in the overview vocabulary 
### (all the words that appear in at least one recipe) and each column represents a recipe

### The TF-IDF score is the frequency of an ingredient occurring in a recipe, down-weighted 
### by the number of recipes in which it occurs.


In [412]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(data['ingredSpaces'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(8110, 5459)

In [413]:
print("5,459 different words were used to describe 8,110 recipes in the dataset")

5,459 different words were used to describe 8,110 recipes in the dataset


Using this matrix we can compute a similarity score. We will use a cosine similarity to calculate a number that denotes the similarity between two recipes.

cosine(x,y0 = x.y^T / ||x||.||y||

Since we have used the TF-IDF vectorizer, calculating the dot product will give us the cosine similarity score. We will use sklearn's linear_kernel() because it is faster.


In [414]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

We're going to define a function that takes in a recipe title as an input and outputs a list of the 10 most similar recipes. 

To do this, we will need a reverse mapping of recipe titles and dataframe indices. This will give us a way to identify the index of a recipe, given its title.

In [415]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(data.index, index=data['ingredSpaces']).drop_duplicates()

indices.head()

ingredSpaces
TacoMeatFilling: groundbeef tacoseasoning Batter: MazolaPure®CookingSpray allpurposeflour masacornflourORcornmeal "envelopesFleischmanns®RapidRiseYeast" sugar salt verywarmmilkdegreesFdegreesF Mazola®CornOil egg Topping: chunkysalsa Mexicanstylecheese cornchips partially    0
groundbeef Pace®TacoSeasoningMix "Campbells®CondensedTomatoSoup" water instantwhiterice Pace®Thick&ChunkySalsa ShreddedCheddarcheese Shreddedlettuce Tortillachips                                                                                                                 1
wholewheatbread groundbeef greenonion  onion salsa wholegrainDijonmustard Worcestershiresauce garlic saltpepper barbequesauce                                                                                                                                                      2
groundbeef onionpowder honeymustard garlicpowder redpepper salt brownsugar oliveoil Swisscheese hamburgerbuns                                               

In [416]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(data['ingredSpaces'])


In [417]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [419]:
# Reset index of our main DataFrame and construct reverse mapping
data = data.reset_index()
indices = pd.Series(data.index, index=data['ingred'])

In [420]:
# Function that takes in ingredient as input and outputs most similar ingredients
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the recipe title that matches the title
    idx = indices[title]

    print("idx[title] = ",idx)
    
    # Get the pairwise similarity scores of all titles with that title
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
 
    # Get the movie indices
    recipe_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar recipes
    return data['title'].iloc[recipe_indices]

In [421]:
get_recommendations(20, cosine_sim2)  #Ann's Dirty Rice


idx[title] =  20


1257                 Ann's Dirty Rice
848                        Porcupines
2085                       Porcupines
236                 Cabbage Rolls III
557               Hamburger Rice Soup
1473                Cabbage Rolls III
1794              Hamburger Rice Soup
3489                   Zucchini Saute
728     Melinda's Porcupine Meatballs
1965    Melinda's Porcupine Meatballs
Name: title, dtype: object

In [422]:
get_recommendations(8083) #Shrimp and Quinoa

idx[title] =  8083


3166                      Grilled Asparagus
3170                     Asparagus Parmesan
2536    Cranberry and Cilantro Quinoa Salad
8030                          Shrimp Quinoa
4300                Spicy and Sweet Spinach
3151                    Pan-Fried Asparagus
7880                  Shrimp and Okra Gumbo
5972       Goat Cheese and Asparagus Quinoa
3520             The Best Steamed Asparagus
5988                 Spiced Quinoa Porridge
Name: title, dtype: object

In [423]:
get_recommendations(8100) #tuna salad with cranberries

idx[title] =  8100


7980                            Quick Tuna Salad
7750                      Avocado and Tuna Tapas
2568                                 Egg Salad I
2543                        Mom's Cucumber Salad
7958                              Halibut Weaver
3568    Asparagus with Cranberries and Pine Nuts
4324                   Creamy Buttered Cucumbers
2473                       Holiday Chicken Salad
2722                       Holiday Chicken Salad
8062                         Cheesy Baked Salmon
Name: title, dtype: object

In [424]:
data['title'].head()

0                   1-Dish Taco Bake
1    15-Minute Dinner Nachos Supreme
2           A Firefighter's Meatloaf
3            Aaron's Missouri Burger
4     Absolute Best Liver and Onions
Name: title, dtype: object

In [425]:
get_recommendations(14) #american lasagne

idx[title] =  14


1251          American Lasagna
6180          American Lasagna
669     Lasagna Stuffed Shells
1906    Lasagna Stuffed Shells
410             Easy Lasagna I
1647            Easy Lasagna I
1164         That's-a Meatloaf
2400         That's-a Meatloaf
1042          Spaghetti Pie II
2278          Spaghetti Pie II
Name: title, dtype: object

In [426]:
get_recommendations(8108) #Mainely Fish

idx[title] =  8108


3354             Summer Vegetable Ratatouille
4087                          Sauteed Cabbage
253                      Carne Asada Marinade
1490                     Carne Asada Marinade
3390                         Roasted Broccoli
3337    Olive Oil Roasted Eggplant with Lemon
7892       Pan-Poached Alaskan Salmon Piccata
7493         Grilled Tilapia with Mango Salsa
4924     Spring Fiddleheads and Sweet Peppers
7857                    Pasta With Tuna Sauce
Name: title, dtype: object

### Next, we'll add more features to help improve recommendations. We'll add Categories and Title.

In [427]:
def clean_categories(ds):
#    for item in ds:
    ds = ds.str.replace(' ','')
    ds = ds.str.lower()
    ds = ds.str.replace('[','')
    ds = ds.str.replace(']','')
    ds = ds.str.replace("''",' ')
    ds = ds.str.replace(',',' ')
    ds = ds.str.replace("'","")
    return ds



In [429]:
data['categories_clean'] = clean_categories(data['categories'])
data['categories_clean'].head()

0    trustedbrands:recipesandtips "argo&#174; karo&...
1      trustedbrands:recipesandtips "campbellskitchen"
2                       maindish meatloaf beefmeatloaf
3                          maindish burgers hamburgers
4                                             maindish
Name: categories_clean, dtype: object

We will use the CountVectorizer() instead of TF-IDF. This is because we do not want to down-weight the presence of an ingredient if it appears in relatively more recipes.

In [432]:
# Merge ingredients and categories into one column
data['ingred_cat'] = data['ingredSpaces'] + ' ' + data['categories_clean'] 


In [433]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(data['ingred_cat'])

In [434]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [435]:
# Reset index of the main DataFrame and construct reverse mapping as before
data = data.reset_index()
indices = pd.Series(data.index, index=data['ingred_cat'])

ValueError: cannot insert level_0, already exists

In [436]:
indices.head()

ingred
TacoMeatFilling:,groundbeef,tacoseasoning,Batter:,MazolaPure®CookingSpray,allpurposeflour,masacornflourORcornmeal,"envelopesFleischmanns®RapidRiseYeast",sugar,salt,verywarmmilkdegreesFdegreesF,Mazola®CornOil,egg,Topping:,chunkysalsa,Mexicanstylecheese,cornchips,partially    0
groundbeef,Pace®TacoSeasoningMix,"Campbells®CondensedTomatoSoup",water,instantwhiterice,Pace®Thick&ChunkySalsa,ShreddedCheddarcheese,Shreddedlettuce,Tortillachips                                                                                                                 1
wholewheatbread,groundbeef,greenonion,,onion,salsa,wholegrainDijonmustard,Worcestershiresauce,garlic,saltpepper,barbequesauce                                                                                                                                                      2
groundbeef,onionpowder,honeymustard,garlicpowder,redpepper,salt,brownsugar,oliveoil,Swisscheese,hamburgerbuns                                                     

In [437]:
get_recommendations(19, cosine_sim2)

idx[title] =  19


1256    Angel's Old Fashioned Beef Stew
827          Pauline Werner's Beef Stew
2064         Pauline Werner's Beef Stew
477                    French Beef Stew
1714                   French Beef Stew
806             Old-Fashioned Beef Stew
2043            Old-Fashioned Beef Stew
600                     Irish Beef Stew
1837                    Irish Beef Stew
979             Slow Cooker Beef Stew I
Name: title, dtype: object

In [441]:
get_recommendations(477, cosine_sim2)

idx[title] =  477


1714                   French Beef Stew
160                   Beer Beef Stew II
568                    Hearty Beef Stew
1397                  Beer Beef Stew II
1805                   Hearty Beef Stew
19      Angel's Old Fashioned Beef Stew
1256    Angel's Old Fashioned Beef Stew
979             Slow Cooker Beef Stew I
2215            Slow Cooker Beef Stew I
6060            Slow Cooker Beef Stew I
Name: title, dtype: object

In [438]:
get_recommendations(8108, cosine_sim2) #Mainely Fish

idx[title] =  8108


8090                                 Spicy Red Snapper
7493                  Grilled Tilapia with Mango Salsa
7791                             Red Snapper Livornese
8026                                 Easiest Spicy Cod
7784                                     Fish Roll-Ups
7857                             Pasta With Tuna Sauce
7745                               Haddock Bubbly Bake
7892                Pan-Poached Alaskan Salmon Piccata
3354                      Summer Vegetable Ratatouille
7781    Pan Fried Halibut Steak with Light Green Sauce
Name: title, dtype: object

In [442]:
get_recommendations(0, cosine_sim2) #1 dish taco bake

idx[title] =  0


1237                                     1-Dish Taco Bake
2766                                    Blackened Chicken
5875                              Mango- Coconut Smoothie
3099    Original Nestle® Toll House Chocolate Chip Coo...
4076                          Creamed Spinach from Oikos®
511                              Grands!® Easy Taco Melts
1748                             Grands!® Easy Taco Melts
5874                          Beet-Red Raspberry Smoothie
596                      Impossibly Easy Cheeseburger Pie
1833                     Impossibly Easy Cheeseburger Pie
Name: title, dtype: object

In [440]:
get_recommendations(8083, cosine_sim2) #Shrimp and Quinoa

idx[title] =  8083


8030                                     Shrimp Quinoa
8040                                      Shrimp Verde
7966              Shrimp with Tomatoes and Feta Cheese
8034                        Lemon Pepper Pasta Seafood
7853                     Lemony Shrimp over Brown Rice
7909               Camarones al Ajillo (Garlic Shrimp)
7833                                Sexy Shrimp Scampi
7681    Fire Roasted Tomato and Feta Pasta with Shrimp
7441                  Garlicky Appetizer Shrimp Scampi
7789                  Garlicky Appetizer Shrimp Scampi
Name: title, dtype: object

## 2 - Recommender Based on Ratings and Number of Reviews

Since ratings can be different based on the number of reviews, for example should a recipe with only 3 ratings, where the average rating is 5.0, be evaluated as having a truly higher rating than a recipe with 500 ratings, where the average rating is about 4.0?<br>

In order to be fair, we are going to use a weighted ratings.<br>

v: number of reviews for the recipe<br>

m: minimum number of votes required to be listed<br>

R: average rating of the recipe<br>

C: mean vote across all recipes<br>

We will use 90th percentile as the cutoff in terms of number of reviews.<br>


In [451]:
# read in fields title, ingredients and categories
dataR = pd.read_csv("dataRecTitleCatsIngredRatRev.csv") 

dataR.duplicated()
dataR.drop_duplicates()

,title,categories,ingred,rating,review_cnt
0,1-Dish Taco Bake,"['Trusted Brands: Recipes and Tips', ""ARGO&#17...","['Taco Meat Filling:', '1 pound ground beef', ...",4.41,86
1,15-Minute Dinner Nachos Supreme,"['Trusted Brands: Recipes and Tips', ""Campbell...","['1 pound ground beef', '1 (1.12 ounce) packag...",4.00,1
2,A Firefighter's Meatloaf,"['Main Dish', 'Meatloaf', 'Beef Meatloaf']","['2 slices whole wheat bread', '2 pounds groun...",4.37,262
3,Aaron's Missouri Burger,"['Main Dish', 'Burgers', 'Hamburgers']","['1 pound lean ground beef', '2 teaspoons onio...",4.67,150
4,Absolute Best Liver and Onions,['Main Dish'],"['2 pounds sliced beef liver', '1 1/2 cups mil...",4.17,9
5,Adobo Sirloin,"['World Cuisine', 'Latin American', 'Mexican']","['1 lime, juiced', '1 tablespoon minced garlic...",4.00,4
6,Albondigas,"['Soups, Stews and Chili', 'Soup', 'Beef Soup']","['1 quart water', '4 carrots, sliced', '2 smal...",4.25,1362
7,Albondigas Soup I,"['Soups, Stews and Chili', 'Soup', 'Pork Soup']","['1 pound lean ground beef', '1/4 pound pork s...",4.28,34
8,Albondigas Soup II,"['Soups, Stews and Chili', 'Soup', 'Beef Soup']","['1 pound lean ground beef', '2 cubes beef bou...",4.00,2
9,Alison's Slow Cooker Vegetable Beef Soup,"['Soups, Stews and Chili', 'Soup', 'Beef Soup']","['1 1/2 pounds cubed beef stew meat', '2 cups ...",4.71,6


In [466]:
C = dataR['rating'].mean()
print("average rating of all recipes: ",C)

average rating of all recipes:  4.158191122071517


In [483]:
m = dataR['review_cnt'].quantile(0.5)
print("50th percentile number of reviews: ",m)

50th percentile number of reviews:  74.0


In [484]:
# Filter out all qualified recipes into a new DataFrame
q_data = dataR.copy().loc[dataR['review_cnt'] >= m]
q_data.shape

(4068, 5)

In [485]:
q_data.duplicated()
q_data.drop_duplicates()

,title,categories,ingred,rating,review_cnt
0,1-Dish Taco Bake,"['Trusted Brands: Recipes and Tips', ""ARGO&#17...","['Taco Meat Filling:', '1 pound ground beef', ...",4.41,86
2,A Firefighter's Meatloaf,"['Main Dish', 'Meatloaf', 'Beef Meatloaf']","['2 slices whole wheat bread', '2 pounds groun...",4.37,262
3,Aaron's Missouri Burger,"['Main Dish', 'Burgers', 'Hamburgers']","['1 pound lean ground beef', '2 teaspoons onio...",4.67,150
6,Albondigas,"['Soups, Stews and Chili', 'Soup', 'Beef Soup']","['1 quart water', '4 carrots, sliced', '2 smal...",4.25,1362
23,Apple Flavored Pot Roast,"['Main Dish', 'Roasts']","['1 (3 pound) boneless beef chuck roast', '1 (...",4.39,137
27,Asian Beef Skewers,['Everyday Cooking'],"['3 tablespoons hoisin sauce', '3 tablespoons ...",4.65,3837
28,Asian Beef with Snow Peas,"['World Cuisine', 'Asian', 'Chinese']","['3 tablespoons soy sauce', '2 tablespoons ric...",4.65,3837
29,Asian Fire Meat,"['World Cuisine', 'Asian', 'Chinese']","['1/2 cup soy sauce', '1 tablespoon sesame oil...",4.64,3837
32,Asian Style Meatloaf,"['World Cuisine', 'Asian']","['1 1/2 pounds ground beef', '1/2 pound ground...",4.47,115
33,Atomic Canuck Chili,"['Soups, Stews and Chili', 'Chili', 'Beef Chili']","['2 pounds lean ground beef', '1/2 large onion...",4.61,325


There are 4068 recipes which qualify to be in this list. Now, we need to calculate our metric for each qualified recipe. To do this, we will define a function, weighted_rating() and define a new feature score, of which we'll calculate the value by applying this function to our DataFrame of qualified recipes:

In [487]:
# Function that computes the weighted rating of each recipe
def weighted_rating(x, m=m, C=C):
    v = x['review_cnt']
    R = x['rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)




In [488]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_data['score'] = q_data.apply(weighted_rating, axis=1)

In [489]:
#Sort movies based on score calculated above
q_data = q_data.sort_values('score', ascending=False)

#Print the top 15 movies
q_data[['title', 'review_cnt', 'rating', 'score']].head(25)

,title,review_cnt,rating,score
1947,Meatball Soup,1023,4.91,4.859285
1948,Meatball Spaghetti Sauce,1019,4.91,4.859100
3203,Dad's Pan-Fried Green Beans,9837,4.84,4.834909
3202,Cream Peas,9836,4.84,4.834909
7021,Pork Chops with Apple Curry Sauce,1390,4.86,4.824526
1232,Zippy Beef Casserole,1777,4.85,4.822343
5600,Coconut Raspberry Smoothie,9933,4.82,4.815106
5601,Chia Ginger Smoothie,9933,4.82,4.815106
5602,Paleo Apple Pie Smoothie,9933,4.82,4.815106
2495,Delicious Egg Salad for Sandwiches,544,4.90,4.811175
